# NML Algorithm 

# Pupil detection (Hough Circle Transform)


### Importing libraries

In [9]:
# libraries

import os, sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install scipy
!{sys.executable} -m pip install opencv-contrib-python
import math
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from pathlib import Path
os.getcwd()
print(cv.__version__)

warcprox 2.4b3.dev192 has requirement urllib3>=1.23, but you'll have urllib3 1.22 which is incompatible.
comet-git-pure 0.19.11 has requirement urllib3>=1.23, but you'll have urllib3 1.22 which is incompatible.
You are using pip version 10.0.1, however version 20.2b1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


warcprox 2.4b3.dev192 has requirement urllib3>=1.23, but you'll have urllib3 1.22 which is incompatible.
comet-git-pure 0.19.11 has requirement urllib3>=1.23, but you'll have urllib3 1.22 which is incompatible.
You are using pip version 10.0.1, however version 20.2b1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


4.2.0


warcprox 2.4b3.dev192 has requirement urllib3>=1.23, but you'll have urllib3 1.22 which is incompatible.
comet-git-pure 0.19.11 has requirement urllib3>=1.23, but you'll have urllib3 1.22 which is incompatible.
You are using pip version 10.0.1, however version 20.2b1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [21]:
# Assuming you have a 'pics' folder

#os.chdir('pics') # relative path: scripts dir is under Lab
# Current working directory (root folder)
cwd = os.getcwd()  

# cascade files 
cascade = cwd + "\\examples\\opencv\\sources\\data\\haarcascades\\haarcascade_frontalface_default.xml'"

# pictures (changed as needed for pictures taken after or before selecting a card)
pic_folder = cwd + "\\before"

opencv_path = cwd + "\\examples\\opencv"

In [22]:
pic_folder

'D:\\Pedro\\Documents\\Cursos\\Radboud\\nmlpython\\before'

#### Funtion to load files

In [12]:
# Load all Pictures (as arrays)
# TODO: Read the pictures in an useful way for the algorithm *RGB
def load_pics_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images



#### Pupil detection method

In [13]:
#global pupil_detected

#eye_cascade = cv.CascadeClassifier('./cascade_files/haarcascade_eye.xml')
def detect_pupils(pic_array):
    global pupil_detected
    itt = 0
    ds_factor = 0.5
    #contours = []
    #global a
    diameter=[]
    #blink = False
    bcount = -1
    
    #kernel = np.ones((3,3),np.uint8)
    kernel = np.ones((5,5),np.uint8)
    circles = []
    font = cv.FONT_HERSHEY_SIMPLEX
    
    for pic in pic_array:
        #pic = cv.resize(pic, None, fx=ds_factor, fy=ds_factor, interpolation=cv.INTER_AREA)
        gray = cv.cvtColor(pic, cv.COLOR_BGR2GRAY)
        #print(gray)
        #eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.3,minNeighbors=1)
        eyes = eye_cascade.detectMultiScale(gray,1.1,7)
        #print(eyes)
        if len(eyes>0):
            for (ex,ey,ew,eh) in eyes:
                roi_gray2 = gray[ey:ey+eh, ex:ex+ew]
                roi_color2 = pic[ey:ey+eh, ex:ex+ew]
                blur = cv.GaussianBlur(roi_gray2,(5,5),10)
                erosion = cv.erode(blur,kernel,iterations = 2)
                ret3,th3 = cv.threshold(erosion,0,255,cv.THRESH_BINARY+cv.THRESH_OTSU)
                circles = cv.HoughCircles(erosion,cv.HOUGH_GRADIENT,4,200,param1=20,param2=150,minRadius=0,maxRadius=0)
                print(type(circles))
                #print(circles)
                if circles is None:
                    print("NO CIRCLES DETECTED!")
                    #cv.imshow(erosion)
                else:
                    for circle in circles[0,:]:
                        if(circle[2]>0 and circle[2]<55):
                            cv.circle(roi_color2,(circle[0],circle[1]),circle[2],(0,0,255),1)
                            cv.putText(pic,"Pupil Pos:",(450,30), font, 1,(0,0,255),2,cv.LINE_AA)
                            cv.putText(pic,"X "+str(int(circle[0]))+" Y "+str(int(circle[1])),(430,60), font, 1,(0,0,255),2,cv.LINE_AA)
                            d = (circle[2]/2.0)
                            dmm = 1/(25.4/d)
                            diameter.append(dmm)
                            cv.putText(pic,str('{0:.2f}'.format(dmm))+"mm",(10,60), font, 1,(0,0,255),2,cv.LINE_AA)
                            cv.circle(roi_color2,(circle[0],circle[1]),2,(0,0,255),3)
                            #pupil_detected = cv.circle(roi_color2,(i[0],i[1]),2,(0,0,255),3)
                            #pic4 = cv2.imshow('erosion',erosion)
                            cv.namedWindow('EyesDetected!')
                            cv.imshow('EyesDetected!', pic)
                            cv.imwrite('./templateA {0}.jpg'.format(itt), pic)
                            cv.imwrite('./templateB {0}.jpg'.format(itt), roi_color2)
                            #cv.imwrite('./templateC {0}.jpg'.format(itt), pic3)
                            cv.waitKey(0)
                        else:
                            print("DIMENSION TROUBLE: CHANGE PARAM")
                            #print("CHANGE SOMETHING PEDRO!")
                            print(circle)
        
        else:
            print("EYES NOT DETECTED")
            print("CHANGE SOMETHING ELSE")
        #cv.namedWindow('EyesDetected!')
        #cv.imshow('EyesDetected!', pupil_detected)
        #cv.imwrite('./template {0}.jpg'.format(i), pupil_detected)
        #cv.waitKey(0)
        itt += 1
        #cap.release()
        #cv.destroyAllWindows()
        #print(pic)
       
       
        

#### Pupil detection. Changed to process more pictures

In [23]:
#global pupil_detected

#eye_cascade = cv.CascadeClassifier('./cascade_files/haarcascade_eye.xml')
def detect_pupils_mod(pic_array):
    global pupil_detected
    itt = 0
    ds_factor = 0.5
    #contours = []
    #global a
    diameter=[]
    #blink = False
    bcount = -1
    
    #kernel = np.ones((3,3),np.uint8)
    kernel = np.ones((5,5),np.uint8)
    circles = []
    font = cv.FONT_HERSHEY_SIMPLEX
    
    for pic in pic_array:
        #pic = cv.resize(pic, None, fx=ds_factor, fy=ds_factor, interpolation=cv.INTER_AREA)
        gray = cv.cvtColor(pic, cv.COLOR_BGR2GRAY)
        #print(gray)
        #eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.3,minNeighbors=1)
        eyes = eye_cascade.detectMultiScale(gray,1.1,7)
        #print(eyes)
       
        for (ex,ey,ew,eh) in eyes:
            roi_gray2 = gray[ey:ey+eh, ex:ex+ew]
            roi_color2 = pic[ey:ey+eh, ex:ex+ew]
            blur = cv.GaussianBlur(roi_gray2,(5,5),10)
            erosion = cv.erode(blur,kernel,iterations = 2)
            ret3,th3 = cv.threshold(erosion,0,255,cv.THRESH_BINARY+cv.THRESH_OTSU)
            circles = cv.HoughCircles(erosion,cv.HOUGH_GRADIENT,4,200,param1=20,param2=150,minRadius=0,maxRadius=0)
            print(type(circles))
            #print(circles)
            if circles is None:
                print("NO CIRCLES DETECTED!")
                #cv.imshow(erosion)
            else:
                for circle in circles[0,:]:
                    if(circle[2]>0 and circle[2]<55):
                        cv.circle(roi_color2,(circle[0],circle[1]),circle[2],(0,0,255),1)
                        cv.putText(pic,"Pupil Pos:",(450,30), font, 1,(0,0,255),2,cv.LINE_AA)
                        cv.putText(pic,"X "+str(int(circle[0]))+" Y "+str(int(circle[1])),(430,60), font, 1,(0,0,255),2,cv.LINE_AA)
                        d = (circle[2]/2.0)
                        dmm = 1/(25.4/d)
                        diameter.append(dmm)
                        cv.putText(pic,str('{0:.2f}'.format(dmm))+"mm",(10,60), font, 1,(0,0,255),2,cv.LINE_AA)
                        cv.circle(roi_color2,(circle[0],circle[1]),2,(0,0,255),3)
                        #pupil_detected = cv.circle(roi_color2,(i[0],i[1]),2,(0,0,255),3)
                        #pic4 = cv2.imshow('erosion',erosion)
                        cv.namedWindow('EyesDetected!')
                        cv.imshow('EyesDetected!', pic)
                        cv.imwrite('./templateA {0}.jpg'.format(itt), pic)
                        cv.imwrite('./templateB {0}.jpg'.format(itt), roi_color2)
                        #cv.imwrite('./templateC {0}.jpg'.format(itt), pic3)
                        cv.waitKey(0)
                    else:
                        print("DIMENSION TROUBLE: CHANGE PARAM")
                        #print("CHANGE SOMETHING PEDRO!")
                        print(circle)
        
        
        #cv.namedWindow('EyesDetected!')
        #cv.imshow('EyesDetected!', pupil_detected)
        #cv.imwrite('./template {0}.jpg'.format(i), pupil_detected)
        #cv.waitKey(0)
        itt += 1
        #cap.release()
        #cv.destroyAllWindows()
        #print(pic)
       

## Usage example

##### Define cascade file 

In [24]:
face_cascade = cv.CascadeClassifier('./cascades/haarcascade_frontalface_default.xml')
eye_cascade = cv.CascadeClassifier('./cascades/haarcascade_eye.xml')

##### Load pictures into array

In [25]:
my_pics = load_pics_from_folder(pic_folder)

In [19]:
my_pics

[array([[[197, 203, 210],
         [197, 203, 210],
         [197, 203, 210],
         ...,
         [169, 177, 184],
         [169, 177, 184],
         [169, 177, 184]],
 
        [[197, 203, 210],
         [197, 203, 210],
         [197, 203, 210],
         ...,
         [169, 177, 184],
         [169, 177, 184],
         [169, 177, 184]],
 
        [[197, 203, 210],
         [197, 203, 210],
         [197, 203, 210],
         ...,
         [169, 177, 184],
         [169, 177, 184],
         [169, 177, 184]],
 
        ...,
 
        [[ 45,  26,  23],
         [ 45,  26,  23],
         [ 45,  26,  23],
         ...,
         [ 28,  17,  13],
         [ 28,  17,  13],
         [ 28,  17,  13]],
 
        [[ 45,  26,  23],
         [ 45,  26,  23],
         [ 45,  26,  23],
         ...,
         [ 28,  17,  13],
         [ 28,  17,  13],
         [ 28,  17,  13]],
 
        [[ 45,  26,  23],
         [ 45,  26,  23],
         [ 45,  26,  23],
         ...,
         [ 28,  17,  13],
  

##### Apply method (output pictures will be generated on the current working directory folder)

In [26]:
detect_pupils_mod(my_pics)

<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'numpy.ndarray'>
DIMENSION TROUBLE: CHANGE PARAM
[58.  46.  58.4]
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'numpy.ndarray'>
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'numpy.ndarray'>
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'numpy.ndarray'>
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>

<class 'numpy.ndarray'>
DIMENSION TROUBLE: CHANGE PARAM
[ 98.  114.   62.8]
<class 'numpy.ndarray'>
DIMENSION TROUBLE: CHANGE PARAM
[206.  194.  102.8]
DIMENSION TROUBLE: CHANGE PARAM
[366.  54. 162.]
DIMENSION TROUBLE: CHANGE PARAM
[ 14.    6.  214.8]
<class 'numpy.ndarray'>
DIMENSION TROUBLE: CHANGE PARAM
[210.  186.   95.2]
<class 'numpy.ndarray'>
DIMENSION TROUBLE: CHANGE PARAM
[102.   62.   87.2]
<class 'numpy.ndarray'>
DIMENSION TROUBLE: CHANGE PARAM
[174.  226.  100.8]
DIMENSION TROUBLE: CHANGE PARAM
[350. 106. 112.]
DIMENSION TROUBLE: CHANGE PARAM
[ 42.   34.  187.6]
<class 'numpy.ndarray'>
DIMENSION TROUBLE: CHANGE PARAM
[194.  142.  104.8]
<class 'numpy.ndarray'>
DIMENSION TROUBLE: CHANGE PARAM
[ 70.  110.   68.8]
<class 'numpy.ndarray'>
DIMENSION TROUBLE: CHANGE PARAM
[146.   98.   76.8]
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'numpy.ndarray'>
DIMENSION TROUBLE: CHANGE PARAM
[226. 194.  92.]
DIMENSION TROUBLE: CHANGE PARAM
[ 14.   46.  321.2]
DIMENSION TROUBLE: CHANGE

<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'numpy.ndarray'>
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType

<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'numpy.ndarray'>
DIMENSION TROUBLE: CHANGE PARAM
[158.  246.  153.2]
<class 'numpy.ndarray'>
DIMENSION TROUBLE: CHANGE PARAM
[138.  210.  116.4]
<class 'numpy.ndarray'>
DIMENSION TROUBLE: CHANGE PARAM
[198.  282.  181.6]
DIMENSION TROUBLE: CHANGE PARAM
[ 78. 110.  88.]
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'numpy.ndarray'>
DIMENSION TROUBLE: CHANGE PARAM
[150.  190.   68.4]
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'numpy.ndarray'>
DIMENSION TROUBLE: CHANGE PARAM
[126.   94.   74.4]
<class 'numpy.ndarray'>
DIMENSION TROUBLE: CHANGE PARAM
[166. 278. 196.]
DIMENSION TROUBLE: CHANGE PARAM
[ 82.   26.  182.4]
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class

<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'numpy.ndarray'>
DIMENSION TROUBLE: CHANGE PARAM
[158.  278.  175.2]
DIMENSION TROUBLE: CHANGE PARAM
[122.   18.  185.6]
DIMENSION TROUBLE: CHANGE PARAM
[318.  102.   88.8]
<class 'numpy.ndarray'>
DIMENSION TROUBLE: CHANGE PARAM
[166. 190. 106.]
DIMENSION TROUBLE: CHANGE PARAM
[342.  290.  169.6]
DIMENSION TROUBLE: CHANGE PARAM
[ 38.  354.  156.4]
<class 'numpy.ndarray'>
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>


<class 'numpy.ndarray'>
DIMENSION TROUBLE: CHANGE PARAM
[234.  186.   74.4]
DIMENSION TROUBLE: CHANGE PARAM
[ 14.  134.  123.2]
DIMENSION TROUBLE: CHANGE PARAM
[ 34. 358. 264.]
DIMENSION TROUBLE: CHANGE PARAM
[350.  366.  180.4]
<class 'numpy.ndarray'>
DIMENSION TROUBLE: CHANGE PARAM
[158.  262.  118.8]
DIMENSION TROUBLE: CHANGE PARAM
[398.  366.  225.2]
DIMENSION TROUBLE: CHANGE PARAM
[326.   26.  181.2]
DIMENSION TROUBLE: CHANGE PARAM
[ 34.  102.   97.6]
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>


<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'numpy.ndarray'>
DIMENSION TROUBLE: CHANGE PARAM
[206. 322. 166.]
DIMENSION TROUBLE: CHANGE PARAM
[334.   22.  227.2]
DIMENSION TROUBLE: CHANGE PARAM
[ 10.  378.  306.4]
DIMENSION TROUBLE: CHANGE PARAM
[  6.   38.  200.8]
<class 'numpy.ndarray'>
DIMENSION TROUBLE: CHANGE PARAM
[238. 218.  90.]
DIMENSION TROUBLE: CHANGE PARAM
[ 42.  146.  140.4]
DIMENSION TROUBLE: CHANGE PARAM
[322.  410.  175.2]
DIMENSION TROUBLE: CHANGE PARAM
[414.   70.  211.2]
DIMENSION TROUBLE: CHANGE PARAM
[ 18. 422. 242.]
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneT

<class 'numpy.ndarray'>
DIMENSION TROUBLE: CHANGE PARAM
[162. 118.  92.]
<class 'numpy.ndarray'>
DIMENSION TROUBLE: CHANGE PARAM
[154.  226.  180.4]
<class 'numpy.ndarray'>
DIMENSION TROUBLE: CHANGE PARAM
[218.  202.   84.8]
DIMENSION TROUBLE: CHANGE PARAM
[354. 354. 346.]
DIMENSION TROUBLE: CHANGE PARAM
[ 50.  382.  333.2]
DIMENSION TROUBLE: CHANGE PARAM
[ 10.  166.  139.6]
DIMENSION TROUBLE: CHANGE PARAM
[406.   78.  152.4]
<class 'numpy.ndarray'>
DIMENSION TROUBLE: CHANGE PARAM
[258. 170. 106.]
DIMENSION TROUBLE: CHANGE PARAM
[154.  350.  245.2]
DIMENSION TROUBLE: CHANGE PARAM
[ 42.   58.  138.8]
<class 'numpy.ndarray'>
DIMENSION TROUBLE: CHANGE PARAM
[210. 130. 112.]
DIMENSION TROUBLE: CHANGE PARAM
[ 74.  282.   93.6]
DIMENSION TROUBLE: CHANGE PARAM
[278.  322.  124.8]
<class 'numpy.ndarray'>
DIMENSION TROUBLE: CHANGE PARAM
[206. 186.  74.]
DIMENSION TROUBLE: CHANGE PARAM
[  6.  242.  237.2]
DIMENSION TROUBLE: CHANGE PARAM
[326. 346. 180.]
<class 'NoneType'>
NO CIRCLES DETECTED!
<c

<class 'numpy.ndarray'>
DIMENSION TROUBLE: CHANGE PARAM
[162.  150.  142.8]
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'numpy.ndarray'>
DIMENSION TROUBLE: CHANGE PARAM
[214.  114.  118.4]
DIMENSION TROUBLE: CHANGE PARAM
[206. 318.  84.]
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'numpy.ndarray'>
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'numpy.ndarray'>
DIMENSION TROUBLE: CHANGE PARAM
[258.  230.  229.2]
DIMENSION TROUBLE: CHANGE PARAM
[138. 398.  66.]
DIMENSION TROUBLE: CHANGE PARAM
[114.   90.   59.6]
DIMENSION TROUBLE: CHANGE PARAM
[346.  422.   81.6]
DIMENSION TROUBLE: CHANGE PARAM
[458.    6.  138.8]
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'numpy.ndarray'>
<class 'NoneType'>
NO CIRCLES DETECTED!
<class 'numpy.ndarray'>
DIMENSION TROUBLE: CHANGE PARAM
[246.  222.  190.4]
DIMENSION TROUBLE: CHANGE PARAM
[542.   78.  449.2]
DIMENSION TROUBLE: CHANGE PARAM
[298.  430.  121.6]
DIMENSION TROUBLE: CHANGE PARAM
[542. 522. 480.]
D

### Other methods implemented. Usage is the same as the examples above


# Face Detection

In [1]:
def detect_faces(pic_array):
    i = 0
    for pic in pic_array:
        gray = cv.cvtColor(pic, cv.COLOR_BGR2GRAY)
        #faces = face_cascade.detectMultiScale(gray, 1.08, 5)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5, minSize=(120, 120))
        for (x, y, w, h) in faces:
            pic = cv.rectangle(pic, (x, y), (x+w, y+h), (255, 0, 0), 2)
            roi_gray = gray[y:y+h, x:x+w]
            eyes = eye_cascade.detectMultiScale(roi_gray, 1.03, 5, minSize=(40, 40))
            for (ex, ey, ew, eh) in eyes:
                pic = cv.rectangle(pic, (x+ex, y+ey),(x+ex+ew, y+ey+eh), (0, 255, 0), 2)
        cv.namedWindow('FaceDetected!')
        cv.imshow('FaceDetected!', pic)
        #cv.SaveImage('pic{:>05}.jpg'.format(i), img)
        cv.imwrite('./template {0}.jpg'.format(i), pic)
        #cv.imwrite('./woodcutters_detected.jpg', pic)
        cv.waitKey(0)
        i += 1

# Pupil detection (Thresholding & Minimum Bounding Rectangle)

In [2]:
def locate_pupils(pic_array):
    i = 0
    ds_factor = 0.5
    contours = []
    for pic in pic_array:
        #pic = cv.resize(pic, None, fx=ds_factor, fy=ds_factor, interpolation=cv.INTER_AREA)
        pic = cv.resize(pic, None, fx=ds_factor, fy=ds_factor, interpolation=cv.INTER_AREA)
        gray = cv.cvtColor(pic, cv.COLOR_BGR2GRAY)
        eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.3,minNeighbors=1)
        for (x_eye, y_eye, w_eye, h_eye) in eyes:
            pupil_frame = gray[y_eye:y_eye + h_eye, x_eye:x_eye + w_eye]
            ret, thresh = cv.threshold(pupil_frame, 80, 255, cv.THRESH_BINARY)
            #ret, thresh = cv2.threshold(pupil_frame, 80, 255, cv2.THRESH_BINARY)
            cv.imshow("threshold", thresh)
            #im2, contours, hierarchy = cv.findContours(thresh, cv.RETR_LIST,cv.CHAIN_APPROX_SIMPLE)
            contours, hierarchy = cv.findContours(thresh, cv.RETR_LIST,cv.CHAIN_APPROX_SIMPLE)
            #print(contours)
            for contour in contours:
                area = cv.contourArea(contour)
                # function to obtain the coordinates of the bounding rectangle
                rect = cv.boundingRect(contour)
                x, y, w, h = rect
                radius = 0.15 * (w + h)
            
                area_condition = (100 <= area <= 200)
                symmetry_condition = (abs(1 - float(w)/float(h)) <= 0.2)
                fill_condition = (abs(1 - (area / (math.pi * math.pow(radius, 2.0))))<= 0.4)
                pupil_det = cv.circle(pic, (int(x_eye + x + radius), int(y_eye + y + radius)),int(1.3 * radius), (0, 180, 0), -1)
                #cv.imshow('Pupil Detector', pic)
                #c = cv2.waitKey(1)
                #if c == 27:
                #    break
             #print(pic)
        cv.namedWindow('EyesDetected!')
        cv.imshow('EyesDetected!', pupil_det)
        cv.imwrite('./template {0}.jpg'.format(i), pupil_det)
        cv.waitKey(0)
        i += 1
    #cap.release()
        #cv.destroyAllWindows()
        #print(pic)
       
        
        
        
        


In [ ]:
# Load Images (as arrays)  by format
#import glob
#ext = ['png', 'jpg', 'gif', 'jpeg']    # Add image formats here
#files = []
#[files.extend(glob.glob(cwd + '\*.' + e)) for e in ext]
#images = [cv.imread(file) for file in files]
#images